In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pytest
import ipytest
ipytest.autoconfig()  # Configurar ipytest para usar pytest en Jupyter Notebook

In [2]:
# Leer el dataset desde un archivo CSV
dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") # cambiar la ruta al probar

# Crear un DataFrame a partir del dataset leído
dataset_df = pd.DataFrame(dataset, columns=dataset.columns)

# Convertir la columna "Grade" a tipo categórico
dataset_df["Grade"] = dataset_df["Grade"].astype('category')

In [3]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Carga el dataset y realiza la división en conjuntos de entrenamiento y prueba."""
        # Leer el dataset desde un archivo CSV
        dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") # cambiar la ruta al probar
        
        # Crear un DataFrame a partir del dataset leído
        self.frame = pd.DataFrame(dataset, columns=dataset.columns)
        
        # Convertir la columna "Grade" a tipo categórico
        self.frame["Grade"] = self.frame["Grade"].astype('category')
        
        # Obtener los nombres de las características excluyendo la columna 'Grade'
        feature_names = [col for col in self.frame.columns if col != 'Grade']
        
        # Dividir el dataset en conjuntos de entrenamiento y prueba
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[feature_names], self.frame['Grade'], test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        """Entrenar el modelo usando el algoritmo especificado (por defecto LogisticRegression)."""
        # Inicializar el modelo con el algoritmo especificado
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        
        # Entrenar el modelo con los datos de entrenamiento
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        """Realizar predicciones usando el modelo entrenado."""
        # Predecir los resultados para los datos de entrada
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        """Obtener la precisión del modelo en el conjunto de prueba."""
        # Calcular y devolver la precisión del modelo en los datos de prueba
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Método de ejecución para correr el pipeline varias veces."""
        # Cargar el dataset y entrenar el modelo
        self.load_dataset()
        self.train()

In [4]:
class PipelineWithFeatureEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        self.scaler = StandardScaler()
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        # Aplica la normalización a los conjuntos de entrenamiento y prueba
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        # Normaliza los datos de entrada antes de realizar predicciones
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        # Carga el dataset, aplica la normalización y entrena el modelo
        self.load_dataset()
        self.apply_scaler()
        self.train()


Pruebas

Realizaremos dos pruebas diferentes:

    Prueba de referencia: Comparar la precisión del modelo contra un benchmark simple
    Prueba diferencial: Comparar la precisión de un modelo contra el otro

Primero, predeciremos la clase más común.


In [5]:
"""
Cuenta la cantidad de ocurrencias de cada valor en la columna "Grade" del DataFrame `dataset_df`.

Returns:
    pandas.Series: Una serie que contiene el conteo de cada valor único en la columna "Grade".
"""

dataset_df["Grade"].value_counts()


Grade
0    497
1    360
Name: count, dtype: int64

In [6]:
@pytest.fixture
def pipelines():
    # Crear una instancia de la primera versión del pipeline
    pipeline_v1 = SimplePipeline()
    
    # Crear una instancia de la segunda versión del pipeline con ingeniería de características
    pipeline_v2 = PipelineWithFeatureEngineering()
    
    # Ejecutar el pipeline de la primera versión
    pipeline_v1.run_pipeline()
    
    # Ejecutar el pipeline de la segunda versión
    pipeline_v2.run_pipeline()
    
    # Devolver ambas instancias de los pipelines
    return pipeline_v1, pipeline_v2

In [7]:
%%ipytest

def test_accuracy_higher_than_benchmark(pipelines):
    pipeline_v1, _ = pipelines
    
    # Benchmark inicial: Predicciones de la clase más común
    benchmark_predictions = [1.0] * len(ppipeline_v1.y_test)
    benchmark_accuracy = accuracy_score(y_true=pipeline_v1.y_test, y_pred=benchmark_predictions)
    
    # Obtener la precisión del modelo
    predictions = pipeline_v1.predict(pipeline_v1.X_test)
    actual_accuracy = accuracy_score(y_true=pipeline_v1.y_test, y_pred=predictions)
    
    # Imprimir la precisión del modelo y del benchmark
    print(f'Accuracy of model 1: {actual_accuracy}, Accuracy of Benchmark: {benchmark_accuracy}')
    
    # Comparar la precisión del primer modelo contra el benchmark
    assert actual_accuracy > benchmark_accuracy

.                                                                                            [100%]
1 passed in 0.08s


In [8]:
%%ipytest

def test_accuracy_compared_to_previous_version(pipelines):
    """
    Compara la precisión de dos versiones de un pipeline de modelos.
    Args:
        pipelines (tuple): Una tupla que contiene dos pipelines de modelos, 
                           donde el primer elemento es la versión 1 y el segundo 
                           elemento es la versión 2.
    Raises:
        AssertionError: Si la precisión de la versión 2 es menor que la precisión 
                        de la versión 1.
    Comentarios:
        - Obtiene la precisión de cada versión del pipeline.
        - Imprime la precisión de cada modelo.
        - Compara la precisión del segundo modelo contra el primero.
    """
    pipeline_v1, pipeline_v2 = pipelines
    
    # Obtener la precisión de cada versión
    v1_accuracy = pipeline_v1.get_accuracy()
    v2_accuracy = pipeline_v2.get_accuracy()
    
    # Imprimir la precisión de cada modelo
    print(f'Accuracy of model 1: {v1_accuracy}')
    print(f'Accuracy of model 2: {v2_accuracy}')
    
    # Comparar la precisión del segundo modelo contra el primero
    assert v2_accuracy >= v1_accuracy

.                                                                                            [100%]
1 passed in 0.06s
